<a href="https://colab.research.google.com/github/SidtheKidx/tone-classification-CadenceIQ/blob/main/tone-classification-main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tone Classification-CadenceIQ using BERT Transformers


## Dataset :


*   [Enron Dataset](https://www.kaggle.com/wcukierski/enron-email-dataset)
*   [Emotions Dataset](https://www.kaggle.com/praveengovi/emotions-dataset-for-nlp?select=val.txt)



In [1]:
import json
!pip install --upgrade "ibm-watson>=5.1.0"

from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

import pandas as pd 
import numpy as np
import os, sys, email, re

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

Requirement already up-to-date: ibm-watson>=5.1.0 in /usr/local/lib/python3.7/dist-packages (5.1.0)


In [2]:
'''emails = pd.read_csv('emails.csv')'''

"emails = pd.read_csv('emails.csv')"

In [3]:
'''emails.head()'''

'emails.head()'

In [4]:
'''emails.shape'''

'emails.shape'

In [5]:
'''emails.message[1]'''

'emails.message[1]'

In [6]:
'''emails.info()'''

'emails.info()'

In [7]:
'''emails.describe()'''

'emails.describe()'

## Data Preprocessing

In [8]:
#To get the content from email objects
'''
def get_text_from_email(msg):
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append(part.get_payload())
    return ''.join(parts) 
    '''

"\ndef get_text_from_email(msg):\n    parts = []\n    for part in msg.walk():\n        if part.get_content_type() == 'text/plain':\n            parts.append(part.get_payload())\n    return ''.join(parts) \n    "

In [9]:
#To separate multiple email addresses
'''
def split_email_addresses(line):
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    
    else:
        addrs = None
    return addrs '''

"\ndef split_email_addresses(line):\n    if line:\n        addrs = line.split(',')\n        addrs = frozenset(map(lambda x: x.strip(), addrs))\n    \n    else:\n        addrs = None\n    return addrs "

In [10]:
# Parse the emails into a list email objects
'''
messages = list(map(email.message_from_string, emails['message']))
emails.drop('message', axis=1, inplace=True)

# Get fields from parsed email objects
keys = messages[0].keys()

for key in keys:
    emails[key] = [doc[key] for doc in messages]
    
# Parse content from emails
emails['content'] = list(map(get_text_from_email, messages))

# Split multiple email addresses
emails['From'] = emails['From'].map(split_email_addresses)
emails['To'] = emails['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails['user'] = emails['file'].map(lambda x:x.split('/')[0])
del messages

emails.head(10) '''

"\nmessages = list(map(email.message_from_string, emails['message']))\nemails.drop('message', axis=1, inplace=True)\n\n# Get fields from parsed email objects\nkeys = messages[0].keys()\n\nfor key in keys:\n    emails[key] = [doc[key] for doc in messages]\n    \n# Parse content from emails\nemails['content'] = list(map(get_text_from_email, messages))\n\n# Split multiple email addresses\nemails['From'] = emails['From'].map(split_email_addresses)\nemails['To'] = emails['To'].map(split_email_addresses)\n\n# Extract the root of 'file' as 'user'\nemails['user'] = emails['file'].map(lambda x:x.split('/')[0])\ndel messages\n\nemails.head(10) "

In [11]:
# Set index and drop columns with two few values
'''
emails = emails.set_index('Message-ID')\
    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)
# Parse datetime
emails['Date'] = pd.to_datetime(emails['Date'], infer_datetime_format=True)
emails.dtypes '''

"\nemails = emails.set_index('Message-ID')    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)\n# Parse datetime\nemails['Date'] = pd.to_datetime(emails['Date'], infer_datetime_format=True)\nemails.dtypes "

In [12]:
#Don't need to clean dataset as IBM watson analyser will do it. Might have to do during MODELLING.

'''
def clean(text):
    stop = set(stopwords.words('english'))
    stop.update(("to", "cc", "subject", "http", "from", "sent"))
    exclude = set(string.punctuation)
    
    #lemmatization is preferred over Stemming because lemmatization does morphological analysis of the words.
    
'''

'\ndef clean(text):\n    stop = set(stopwords.words(\'english\'))\n    stop.update(("to", "cc", "subject", "http", "from", "sent"))\n    exclude = set(string.punctuation)\n    \n    #lemmatization is preferred over Stemming because lemmatization does morphological analysis of the words.\n    \n'

In [13]:
''' emails['content'][10] '''

" emails['content'][10] "

In [14]:
''' tone_df = emails[['content']][:2000] '''

" tone_df = emails[['content']][:2000] "

In [15]:
#new dataset for message content and its tone
#tone_df.to_csv(r'/Users/siddharthsingh/CadenceIQ/tone_Analyzer/tone-content-data.csv')
#tone_df = pd.read_csv('tone-content-data.csv')

In [16]:
'''tone_df.head()'''

'tone_df.head()'

In [17]:
'''tone_df.shape'''

'tone_df.shape'

In [18]:
#IBM watson tone analyser API endpoint 
'''
authenticator = IAMAuthenticator('7UkBO29yBlRAuDZzA5s8Nl-IbmlR3mK1yINRazApqitf')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)
tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/cd914da7-f09f-4298-862d-d4c6005ac8b3')
'''

"\nauthenticator = IAMAuthenticator('7UkBO29yBlRAuDZzA5s8Nl-IbmlR3mK1yINRazApqitf')\ntone_analyzer = ToneAnalyzerV3(\n    version='2017-09-21',\n    authenticator=authenticator\n)\ntone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/cd914da7-f09f-4298-862d-d4c6005ac8b3')\n"

In [19]:
#Converting column to list. We could also use df['column'].tolist() OR list(tone_df['content'])
'''
text = list(tone_df['content'])
text '''

"\ntext = list(tone_df['content'])\ntext "

In [20]:
'''c = 0
list_tones = [] '''

'c = 0\nlist_tones = [] '

In [21]:
'''
for i in text:
    print(i)
    tone_analysis = tone_analyzer.tone(
        {'text': i},
        content_type='application/json'
    ).get_result()
    
    list_tones.append(json.dumps(tone_analysis, indent=2))
    c+=1
    
tone_df = tone_df.assign(tones = list_tones)
#print(json.dumps(tone_analysis, indent=2))     '''

"\nfor i in text:\n    print(i)\n    tone_analysis = tone_analyzer.tone(\n        {'text': i},\n        content_type='application/json'\n    ).get_result()\n    \n    list_tones.append(json.dumps(tone_analysis, indent=2))\n    c+=1\n    \ntone_df = tone_df.assign(tones = list_tones)\n#print(json.dumps(tone_analysis, indent=2))     "

In [22]:
'''tone_df.head()'''

'tone_df.head()'

In [23]:
'''tone_df.to_csv(r'/Users/siddharthsingh/CadenceIQ/tone_Analyzer/tone-content-data-og.csv')'''

"tone_df.to_csv(r'/Users/siddharthsingh/CadenceIQ/tone_Analyzer/tone-content-data-og.csv')"

In [24]:
#print(json.dumps(tone_analysis, indent=2)) 

In [25]:
#print(tone_analysis['sentences_tone'][1]['tones'][0])

## Accessing JSON data to visualize distribution of Enron dataset tones

In [26]:
og_tone = pd.read_csv('https://raw.githubusercontent.com/SidtheKidx/tone-classification-CadenceIQ/main/tone-content-data-og.csv?token=AJVH2H3VF6XLJSAM55EQ5ILATE5JM')

HTTPError: ignored

In [ ]:
og_tone['tones'][5]

In [ ]:
import json

In [ ]:
#To replce \\n with \n. Helps in visaualising JSON data.
og_tone['tones'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [ ]:
#Any content with just ONE sentence does NOT have a sentence-id in JSON data.
og_tone.head(10)

In [ ]:
og_tone['tones'][5]

In [ ]:
json_data = pd.read_json('https://raw.githubusercontent.com/SidtheKidx/tone-classification-CadenceIQ/main/ibm-tones.json?token=AJVH2H3633D2MWP2YHF4YODATE5RI', lines = True)

In [ ]:
counter_dict = {
    
        'anger' : {
            'score' : 0,
            'count' : 0
        }, 
        'analytical' : {
            'score' : 0,
            'count' : 0
        },
        'fear' :{
            'score' : 0,
            'count' : 0
        },
        'joy' : {
            'score' : 0,
            'count' : 0
        },
        'sadness' : {
            'score' : 0,
            'count' : 0
        },
        'confident' : {
            'score' : 0,
            'count' : 0
        },
        'tentative' : {
            'score' : 0,
            'count' : 0
        }
}

for json_data in og_tone['tones']:
    data_dict = json.loads(str(json_data))
    
    for document_tones in data_dict['document_tone']['tones']:
        counter_dict[document_tones['tone_id']]['count']+=1
        counter_dict[document_tones['tone_id']]['score']= (counter_dict[document_tones['tone_id']]['score'] + document_tones['score'])/2

print(counter_dict)

### Visualizating Enron tones as a bar plot


In [ ]:
#The dictionary of tones, count & score converted to DATAFRAME for visualization
vis_df = pd.DataFrame.from_dict(counter_dict)
vis_ax = vis_df.transpose()
vis_ax.reset_index(level=0, inplace=True)
print(vis_ax)
vis_ax['count'].sum()

### Distribution of tones

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

sns.barplot(x = vis_ax['index'], y = vis_ax['count'], data=vis_ax).set_title('Enron tone distribution')

In [ ]:
# Indent keyword while dumping the
# data decides to what level 
# spaces the user wants.
#print(json.dumps(data_dict, indent = 1))

#print(data_dict['sentences_tone'])

# Difference in the spaces 
# near the brackets can be seen
#print(json.dumps(data_dict, indent = 3))

In [ ]:
#df_train = pd.read_csv("/Users/siddharthsingh/CadenceIQ/tone_Analyzer/emotions-dataset/train.txt", delimiter=';', header=None, names=['sentence','label'])
#df_test = pd.read_csv("/Users/siddharthsingh/CadenceIQ/tone_Analyzer/emotions-dataset/test.txt", delimiter=';', header=None, names=['sentence','label'])
#df_val = pd.read_csv("/Users/siddharthsingh/CadenceIQ/tone_Analyzer/emotions-dataset/val.txt", delimiter=';', header=None, names=['sentence','label'])

In [ ]:
#emo_df = pd.concat([df_train,df_test,df_val])

In [ ]:
#emo_df['label'].unique()

In [ ]:
#emo_df.head(10)

In [ ]:
#emo_df.describe()

In [ ]:
'''
count = emo_df['label'].value_counts()
print(count)
count.sum()
'''

In [ ]:
#emo_df1 = emo_df[:1500].copy()

In [ ]:
#emo_df1.shape

In [ ]:
#emo_df1.head()

In [ ]:
#emo_df1.describe()

In [ ]:
'''
count_df1 = emo_df1['label'].value_counts()
print(count_df1)
count_df1.sum()
'''

In [ ]:
#sns.countplot(x=count_df1, data = emo_df1['label'])

## Accessing JSON data to visualize distribution of Emotions dataset tones

In [ ]:
#IBM watson tone analyser API endpoint 
'''
authenticator = IAMAuthenticator('SwdiI5vCKkgeyJuYIEwhOkajctEnxTR-F7OYZ4pvF9GE')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)
tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/3e193de0-bcaa-47c0-8898-4c68a520ce78')
'''

In [ ]:
#Converting column to list. 
'''
text = list(emo_df1['sentence'])
text
''' 

In [ ]:
'''
c = 0
list_tones = []


for i in text:
    print(i)
    tone_analysis = tone_analyzer.tone(
        {'text': i},
        content_type='application/json'
    ).get_result()

    list_tones.append(json.dumps(tone_analysis, indent=2))
    c+=1

emo_df1 = emo_df1.assign(tones = list_tones)
#print(json.dumps(tone_analysis, indent=2)) 
'''

In [ ]:
#emo_df1.head(10)

In [ ]:
#emo_df1['sentence'][9]

In [ ]:
#emo_df1['tones'][7]

In [ ]:
#emo_df1['tones'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [ ]:
#emo_df1['tones'][7]

In [ ]:
emo_df1 = pd.read_csv('https://raw.githubusercontent.com/SidtheKidx/tone-classification-CadenceIQ/main/Emotions-tone-data-og.csv?token=AJVH2HZ7S2LTSAWXB2ENDVTATE56A')

In [ ]:
json_df1 = pd.read_json('https://raw.githubusercontent.com/SidtheKidx/tone-classification-CadenceIQ/main/emotions-tones.json?token=AJVH2H2KBQGBQCXXV4ERCETATE6BE', lines=True)

In [ ]:
counter_dict_df1 = {
    
        'anger' : {
            'score' : 0,
            'count' : 0
        }, 
        'analytical' : {
            'score' : 0,
            'count' : 0
        },
        'fear' :{
            'score' : 0,
            'count' : 0
        },
        'joy' : {
            'score' : 0,
            'count' : 0
        },
        'sadness' : {
            'score' : 0,
            'count' : 0
        },
        'confident' : {
            'score' : 0,
            'count' : 0
        },
        'tentative' : {
            'score' : 0,
            'count' : 0
        }
}

for json_df1 in emo_df1['tones']:
    data_dict_df1 = json.loads(str(json_df1))
    
    for document_tones_df1 in data_dict_df1['document_tone']['tones']:
        counter_dict_df1[document_tones_df1['tone_id']]['count']+=1
        counter_dict_df1[document_tones_df1['tone_id']]['score']= (counter_dict_df1[document_tones_df1['tone_id']]['score'] + document_tones_df1['score'])/2

print(counter_dict_df1)

### Visualizing Emotions tones as bar plot

In [ ]:
#The dictionary of tones, count & score converted to DATAFRAME for visualization

vis_df1 = pd.DataFrame.from_dict(counter_dict_df1)
vis_ax1 = vis_df1.transpose()
vis_ax1.reset_index(level=0, inplace=True)
print(vis_ax1)
vis_ax1['count'].sum()

### Distribution of tones

In [ ]:
sns.barplot(x = vis_ax1['index'], y = vis_ax1['count'], data=vis_ax1).set_title('Emotions tone distribution')

## Text encoding & Tokenization for BERT

In [ ]:
emo_df1.head()

In [ ]:
emo_df1['tones'][7]

In [ ]:
missing_values_check = emo_df1.isnull().sum()
print(missing_values_check)

In [ ]:
emo_df1 = emo_df1.drop(['label'], axis=1)

In [ ]:
emo_df1.head()

In [ ]:
'''
# creating initial dataframe
tone_df1 = emo_df1.copy()
tone_types = ('Anger','Analytical','Fear','Joy','Sadness','Confident','Tentative')
tone_df1 = pd.DataFrame(tone_types, columns=['Bridge_Types'])
# generate binary values using get_dummies
dum_df = pd.get_dummies(tone_df1)
# merge with main df emo_df1 on key values
tone_df1 = tone_df1.join(dum_df)
tone_df1
'''

In [ ]:
emo_df2 = emo_df1.copy()
emo_df2.head()

In [ ]:
list_tones = []
c = 0

for json_df1 in emo_df2['tones']:
    fresh_list = [] 
    data_dict_df2 = json.loads(str(json_df1))

    for document_tones_df2 in data_dict_df2['document_tone']['tones']:
      fresh_list.append(document_tones_df2['tone_id'])
      c+=1

    list_tones.append(fresh_list)

emo_df2['tags'] = pd.Series(list_tones)

print(list_tones)
print(c)

In [ ]:
emo_df2.head()